<a href="https://colab.research.google.com/github/HectorRPabon/Colab/blob/main/FIRMS_API_Hotspots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#FIRMS DATA FOR WILDFIRE TIME MONITORING

MODIS: Historical trends and hotspot analysis
Providing data for wildfire monitoring since 2000 with 1km spatial resolution.

Youtube Tutorial [here](https://www.youtube.com/live/iMbfxlNZRSk)
For the data send message to youtube channel [Planet Sapling](https://www.youtube.com/@AnkurShah) or a mail to [Ankur Shah](ankur@planetsapling.com) or his [personal mail](mybuddyankur@gmail.com)

In [ ]:
import pandas as pd
import folium
import geopandas as gpd
from folium.plugins import MarkerCluster

In [ ]:
#FIRM personal registration in NASA
MAP_KEY = '430c52106a780ccb3cd78318ad2dd3b9'
#MAP_KEY = 'bd5373447ecdae7d78f324647e40177e'

IndentationError: unexpected indent (<ipython-input-10-0321c862a40d>, line 3)

In [ ]:
# this url will return information about all supported sensors and their corresponding datasets
# instead of 'all' you can specify individual sensor, ex:LANDSAT_NRT
da_url = 'https://firms.modaps.eosdis.nasa.gov/api/data_availability/csv/' + MAP_KEY + '/all'
df = pd.read_csv(da_url)
display(df)

,data_id,min_date,max_date
0,MODIS_NRT,2024-12-01,2025-03-06
1,MODIS_SP,2000-11-01,2024-11-30
2,VIIRS_NOAA20_NRT,2024-12-01,2025-03-05
3,VIIRS_NOAA20_SP,2018-04-01,2024-11-30
4,VIIRS_NOAA21_NRT,2024-01-17,2025-03-06
5,VIIRS_SNPP_NRT,2025-01-01,2025-03-05
6,VIIRS_SNPP_SP,2012-01-20,2024-12-31
7,LANDSAT_NRT,2022-06-20,2025-03-05
8,GOES_NRT,2022-08-09,2025-03-06
9,BA_MODIS,2000-11-01,2024-11-01


In [ ]:
# Function to fetch MODIS hotspots by country
def get_hotspots_by_country(country_code, day_range):
    url = f'https://firms.modaps.eosdis.nasa.gov/api/country/csv/{MAP_KEY}/MODIS_NRT/{country_code}/{day_range}'
    try:
        df = pd.read_csv(url)
        print(f"Retrieved {len(df)} records for country {country_code}")
        return df
    except:
        print("Error retrieving data for the specified country.")
        return None

In [ ]:
#HOTSPOT COLOMBIA FIRES last 5 Days ***COL/5** could be change to "/VIIRS_NOAA20_NRT/COL/5" , must up to 40 to 75 modern confidence, frp meassure the fire power more energy releaze 35
colombia_url = 'https://firms.modaps.eosdis.nasa.gov/api/country/csv/' + MAP_KEY + '/MODIS_NRT/COL/5'
df_colombia = pd.read_csv(colombia_url)
df_colombia

,country_id,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight
0,COL,3.42238,-72.37173,308.69,1.77,1.30,2025-03-02,158,Terra,MODIS,75,6.1NRT,274.43,21.33,N
1,COL,4.43120,-71.14539,317.44,1.38,1.16,2025-03-02,158,Terra,MODIS,95,6.1NRT,275.11,27.40,N
2,COL,4.43296,-71.13332,303.51,1.38,1.16,2025-03-02,158,Terra,MODIS,55,6.1NRT,273.16,11.31,N
3,COL,4.84052,-71.79161,307.11,1.51,1.21,2025-03-02,158,Terra,MODIS,70,6.1NRT,293.79,10.24,N
4,COL,4.86811,-71.67509,310.54,1.48,1.20,2025-03-02,158,Terra,MODIS,80,6.1NRT,294.74,13.05,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,COL,11.07913,-72.91452,348.71,1.41,1.18,2025-03-05,1902,Aqua,MODIS,94,6.1NRT,300.61,87.53,D
507,COL,11.08103,-72.90239,319.91,1.41,1.18,2025-03-05,1902,Aqua,MODIS,69,6.1NRT,300.27,18.52,D
508,COL,11.08548,-72.92204,331.50,1.41,1.18,2025-03-05,1902,Aqua,MODIS,83,6.1NRT,301.03,37.98,D
509,COL,11.08752,-72.90952,341.99,1.41,1.18,2025-03-05,1902,Aqua,MODIS,90,6.1NRT,300.99,65.33,D


In [ ]:
df = df_colombia

In [ ]:
# Initialize Folium Map
m = folium.Map(location=[df["latitude"].mean(), df["longitude"].mean()], zoom_start=6)
marker_cluster = MarkerCluster().add_to(m)

# Add fire markers
for _, row in df.iterrows():
    popup_text = (f"<b>Brightness Temp:</b> {row['brightness']}<br>"
                  f"<b>Confidence:</b> {row['confidence']}<br>"
                  f"<b>FRP:</b> {row['frp']}<br>"
                  f"<b>Date:</b> {row['acq_date']}<br>"
                  f"<b>Satellite:</b> {row['satellite']}")
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=folium.Popup(popup_text, max_width=300),
        icon=folium.Icon(color="red", icon="fire")
    ).add_to(marker_cluster)

# Display the map
m